In [1]:
import twnews_consumer
reload(twnews_consumer)

from twnews_consumer import main


In [2]:
import logging
logging.info('123')

2016-03-27 19:04:23: 123


In [174]:
from twnews_consumer.twitter_fetcher.fetcher import TwitterFetcher
reload(twnews_consumer.twitter_fetcher.fetcher)
from twnews_consumer import defaults
print defaults.RSS_DB_PATH
tf = TwitterFetcher()

/tmp/twnews_data/logs/rss.shelve


In [175]:
stream = tf.create_tweepy_stream()

In [176]:
#stream.sample()

In [211]:
tweets_data = open('/tmp/test.tweets').read()
tweets = filter(None, tweets_data.splitlines())
tweets = map(json.loads, tweets)

In [232]:
def is_en(tweet):
    return 'delete' not in tweet and tweet['lang'] == 'ru'

def parse_tweet(source_text):
    tweet = source_text.copy()
    if 'delete' not in tweet:
        # info about base_tweet
        created_at = tweet['created_at']
        retweeted = 'retweeted_status' in tweet        
        
        # go deeper :-)
        while 'retweeted_status' in tweet:
            tweet = tweet['retweeted_status']
        
        # info about dest tweet
        lang = tweet['lang']
        entities = tweet['entities']
        text = tweet['text']
        
        #extract links and media links        
        media_links = {}
        if 'media' in entities:
            for media in entities['media']:
                media_links[media['url']] = media['expanded_url']                
        links = {}
        if 'urls' in entities:
            for url in entities['urls']:
                links[url['url']] = url['expanded_url']
        
        #extract hastags
        hashtags = {}
        if 'hashtags' in tweet['entities']:
            for hashtag in tweet['entities']['hashtags']:
                hashtags[hashtag['text']] = hashtag['indices']        
        
        #remove url of all links from the text
        for url in media_links.keys() + links.keys():
            text = text.replace(url, '')
        
        return {
            'retweeted': retweeted,
            'created_at': created_at,
            'lang': lang,
            'text': text,            
            'links': links,
            'meida_links': media_links,
            'hashtags': hashtags
        }
#reload(parse_tweet)

In [233]:
en_tweets = filter(is_en,tweets)
errors = []
for tweet in en_tweets:
    try:
        parsed = parse_tweet(tweet)
    except Exception as e:
        errors.append(tweet)
#parsed_tweets = map(parse_tweet, en_tweets)

In [240]:
#parse_tweet(en_tweets[0])
print len(errors), len(en_tweets)
#parse_tweet(errors[0])
en_tweets[0]

0 34


{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Sun Mar 27 16:11:26 +0000 2016',
 u'entities': {u'hashtags': [],
  u'symbols': [],
  u'urls': [],
  u'user_mentions': []},
 u'favorite_count': 0,
 u'favorited': False,
 u'filter_level': u'low',
 u'geo': None,
 u'id': 714122668803551232,
 u'id_str': u'714122668803551232',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'is_quote_status': False,
 u'lang': u'ru',
 u'place': None,
 u'retweet_count': 0,
 u'retweeted': False,
 u'source': u'<a href="http://ifttt.com" rel="nofollow">IFTTT</a>',
 u'text': u'\u0420\u0435\u0446\u0435\u043f\u0442 \u043a\u0430\u043b\u044c\u043c\u0430\u0440\u043e\u0432 \u0432 \u0441\u043c\u0435\u0442\u0430\u043d\u043d\u043e\u043c \u0441\u043e\u0443\u0441\u0435 \u041b\u044e\u0431\u0438\u0442\u0435\u043b\u044f\u043c \u043c\u043e\u0440\u0435\u043f\u0440\u043e\u0434\u0443\u043a\u0442

In [235]:
parse_tweet(en_tweets[1])

{'created_at': u'Sun Mar 27 16:11:26 +0000 2016',
 'hashtags': {},
 'lang': u'ru',
 'links': {},
 'meida_links': {},
 'retweeted': False,
 'text': u'\u0444\u0443\u0445.'}

In [236]:
parse_tweet(en_tweets[2])

{'created_at': u'Sun Mar 27 16:11:27 +0000 2016',
 'hashtags': {},
 'lang': u'ru',
 'links': {},
 'meida_links': {u'https://t.co/gvcGm3oiqu': u'http://twitter.com/elvina0407/status/714122672997855232/photo/1'},
 'retweeted': False,
 'text': u'\u041e\u0442\u043a\u0440\u044b\u0432\u0430\u0435\u043c \u0442\u0430\u043b\u0430\u043d\u0442\u044b \u043d\u0430 \u0444\u0435\u0441\u0442\u0438\u0432\u0430\u043b\u0435 \u0434\u0440\u0430\u043c. \u043a\u043e\u043b\u043b\u0435\u043a\u0442\u0438\u0432\u043e\u0432 \u041f\u041c\u0420, \u043f\u043e\u0441\u0432\u044f\u0449\u0451\u043d\u043d\u043e\u043c \u0413\u043e\u0434\u0443 \u043a\u0438\u043d\u043e. \u0412\u044b\u0441\u0442\u0443\u043f\u043b\u0435\u043d\u0438\u0435 \u043a\u043e\u043b-\u0432\u0430 \u0420\u043e\u0436\u0434\u0435\u0441\u0442\u0432\u0435\u043d\u0441\u043a\u043e\u0433\u043e \u0421\u0414\u041a '}

In [237]:
parse_tweet(en_tweets[3])

{'created_at': u'Sun Mar 27 16:11:29 +0000 2016',
 'hashtags': {u'\u041e\u0431\u0430\u043c\u0430': [107, 113]},
 'lang': u'ru',
 'links': {},
 'meida_links': {u'https://t.co/87z45OyhVH': u'http://twitter.com/vlad_zhukovskiy/status/714100713228513285/photo/1'},
 'retweeted': True,
 'text': u'\u041f\u0438\u043f\u043b \u0445\u0430\u0432\u0430\u0435\u0442: \u0432 \u0440\u043e\u0441\u0442\u0435 \u0446\u0435\u043d \u0438 \u0430\u043a\u0446\u0438\u0437\u043e\u0432 \u043d\u0430 \u0442\u043e\u043f\u043b\u0438\u0432\u043e \u0438 \u0442\u0430\u0440\u0438\u0444\u043e\u0432 \u0416\u041a\u0425, \u0441\u0431\u043e\u0440\u0430\u0445 \u0437\u0430 \u043a\u0430\u043f\u0440\u0435\u043c\u043e\u043d\u0442 \u0438 \u043f\u0430\u0434\u0435\u043d\u0438\u0438 \u0437\u0430\u0440\u043f\u043b\u0430\u0442 \u0432\u0438\u043d\u043e\u0432\u0430\u0442 #\u041e\u0431\u0430\u043c\u0430 '}

In [238]:
parse_tweet(en_tweets[4])

{'created_at': u'Sun Mar 27 16:17:37 +0000 2016',
 'hashtags': {},
 'lang': u'ru',
 'links': {},
 'meida_links': {},
 'retweeted': False,
 'text': u'\u043a\u0430\u043a \u043f\u0435\u0440\u0435\u0441\u043f\u0430\u0442\u044c \u0441\u043e \u0432\u0441\u0435\u043c \u043c\u0438\u0440\u043e\u043c?'}

In [239]:
parse_tweet(en_tweets[5])

{'created_at': u'Sun Mar 27 16:17:38 +0000 2016',
 'hashtags': {},
 'lang': u'ru',
 'links': {},
 'meida_links': {},
 'retweeted': True,
 'text': u'\u041c\u043d\u0435 \u0440\u0430\u043d\u043e \u0437\u0430\u0432\u043e\u0434\u0438\u0442\u044c \u0434\u0435\u0442\u0435\u0439. \u042f \u043c\u043e\u0440\u0430\u043b\u044c\u043d\u043e \u043d\u0435 \u0433\u043e\u0442\u043e\u0432 \u043f\u043e\u043a\u0443\u043f\u0430\u0442\u044c \u043a\u0438\u043d\u0434\u0435\u0440-\u0441\u044e\u0440\u043f\u0440\u0438\u0437 \u0438 \u043e\u0442\u0434\u0430\u0432\u0430\u0442\u044c \u0435\u0433\u043e \u043a\u043e\u043c\u0443-\u0442\u043e.'}

In [6]:
import feedparser

from twnews_consumer import defaults
feeds = defaults.RSS_FEEDS
feed = feedparser.parse(feeds['gazeta']['rss_url'])

In [7]:
feed

{'bozo': 0,
 'encoding': u'windows-1251',
 'entries': [{'author': u'\u0414\u043c\u0438\u0442\u0440\u0438\u0439 \u041a\u0438\u0440\u0438\u043b\u043b\u043e\u0432 (\u041c\u0430\u0440\u0438\u0443\u043f\u043e\u043b\u044c)',
   'author_detail': {'name': u'\u0414\u043c\u0438\u0442\u0440\u0438\u0439 \u041a\u0438\u0440\u0438\u043b\u043b\u043e\u0432 (\u041c\u0430\u0440\u0438\u0443\u043f\u043e\u043b\u044c)'},
   'authors': [{}],
   'guidislink': False,
   'id': u'http://www.gazeta.ru/boeing/politics/2015/07/17_a_7644137.shtml',
   'link': u'http://www.gazeta.ru/boeing/politics/2015/07/17_a_7644137.shtml',
   'links': [{'href': u'http://www.gazeta.ru/boeing/politics/2015/07/17_a_7644137.shtml',
     'rel': u'alternate',
     'type': u'text/html'},
    {'href': u'http://img.gazeta.ru/files3/809/7644809/mediaPic-pic60-60x60-42517.jpg',
     'rel': u'enclosure',
     'type': u'image/jpeg'}],
   'published': u'Sat, 18 Jul 2015 19:46:09 +0300',
   'published_parsed': time.struct_time(tm_year=2015, tm_m

In [245]:
e = feed.entries[0]

In [248]:
'summary' in e

True